In [275]:
import tensorflow as tf
import numpy as np
import pandas as pd
import time
import gc
import matplotlib.pyplot as plt
%matplotlib inline

NameError: name 'roc_auc_score' is not defined

In [150]:
csv = pd.read_csv("train_ver2.csv")
csv.drop("fecha_alta", axis=1, inplace=True)
csv.drop("ind_nuevo", axis=1, inplace=True)
csv.drop("indext", axis=1, inplace=True)

/Users/yury/anaconda/envs/tensorflow/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [151]:
csv["fecha_dato_dt"] = pd.to_datetime(csv["fecha_dato"])
csv["ult_fec_cli_1t"] = pd.to_datetime(csv["ult_fec_cli_1t"])

In [152]:
csv['age'].replace([" NA","NA",""], np.nan, inplace=True)

In [153]:
csv['age'] = csv['age'].astype(np.float64)

In [154]:
csv = csv[np.isfinite(csv['age'])]

In [155]:
csv["ncodpers"] = csv["ncodpers"].astype(np.int32)

In [156]:
csv["ind_empleado"].value_counts()

N    13610977
B        3566
F        2523
A        2492
S          17
Name: ind_empleado, dtype: int64

In [157]:
ind_empleado_vals = [0,1,2,3,4]
csv["ind_empleado"].replace(["N","B","F","A","S"], ind_empleado_vals, inplace=True)
csv["ind_empleado"] = csv["ind_empleado"].astype(np.int32)


In [158]:
csv["pais_residencia"] = [int(b) for b in csv["pais_residencia"] == "ES"]
csv["pais_residencia"] = csv["pais_residencia"].astype(np.int32)

In [159]:
csv["sexo"] = csv["sexo"].apply(lambda s: 1 if s == "H" else 0)
csv["sexo"] = csv["sexo"].astype(np.int32)

In [160]:
csv["age"] = csv["age"].apply(lambda s: 100. if s >100. else s)
csv["age"] = csv["age"].apply(lambda s: s*10 if s < 10. else s)
csv["age"] = csv["age"]/100.

In [161]:
csv["antiguedad"] = csv["antiguedad"].astype(np.int32)

In [162]:
antiguedad_median = csv["antiguedad"].median()
csv["antiguedad"] = csv["antiguedad"].apply(lambda s: antiguedad_median if s <= 0. else s)
csv["antiguedad"] = csv["antiguedad"]/300

In [163]:
csv["indrel"] = csv["indrel"].apply(lambda s: 0 if s == 99. else 1).astype(np.int32)

In [164]:
csv["ult_fec_cli_1t"][csv["indrel"] == 1] = csv["fecha_dato_dt"][csv["indrel"] == 1]

idx = csv["ult_fec_cli_1t"] - csv["fecha_dato_dt"] < np.timedelta64(1, 'D')
csv["ult_fec_cli_1t"][idx] = csv["fecha_dato_dt"][idx]

/Users/yury/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/yury/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [165]:
prim_mon = (csv["ult_fec_cli_1t"] - csv["fecha_dato_dt"]).astype('timedelta64[D]') 
csv["ult_fec_cli_1t"] = prim_mon / 200.

In [167]:
csv.drop("fecha_dato_dt", axis=1, inplace=True)

In [166]:
csv["indrel_1mes"][csv["indrel_1mes"] == "P"] = 5

/Users/yury/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [168]:
csv["indrel_1mes"] = csv["indrel_1mes"].astype(np.float64)
csv["indrel_1mes"][np.isnan(csv["indrel_1mes"])] = 0.
csv["indrel_1mes"] = csv["indrel_1mes"].astype(np.int32)



/Users/yury/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [169]:
csv["indresi"] = csv["indresi"].apply(lambda s: 1 if s == "S" else 0).astype(np.int32)

In [170]:
csv["canal_entrada"].value_counts()[:10]

KHE    4055270
KAT    3268209
KFC    3098360
KHQ     591039
KFA     409669
KHK     241084
KHM     183924
KHD     116891
KHN     116608
KAS      86221
Name: canal_entrada, dtype: int64

In [171]:
tiprel_1mes_vals = [0,1,2,3,4]
csv["tiprel_1mes"].replace(["I","A","P","R","N"], tiprel_1mes_vals, inplace=True)
csv["tiprel_1mes"][np.isnan(csv["tiprel_1mes"])] = 0.
csv["tiprel_1mes"] = csv["tiprel_1mes"].astype(np.int32)

/Users/yury/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [172]:
csv.drop("conyuemp", axis=1, inplace=True)
#csv["conyuemp"] = csv["conyuemp"].apply(lambda s: 1 if s == "S" else 0).astype(np.int32)

In [173]:
canal_entrada_names = ["OTHER", "KHE","KAT","KFC", "KHQ", "KFA" ,"KHK" ,"KHM", "KHD", "KHN"]
canal_entrada_vals = list(range(len(canal_entrada_names)))

csv["canal_entrada"].replace(canal_entrada_names, canal_entrada_vals, inplace=True)
csv["canal_entrada"] = csv["canal_entrada"].apply(lambda s: 0 if isinstance(s, str) else s)
csv["canal_entrada"][np.isnan(csv["canal_entrada"])] = 0

/Users/yury/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [174]:
csv["canal_entrada"] = csv["canal_entrada"].astype(np.int32)

In [175]:
csv["indfall"].value_counts()

N    13584813
S       34762
Name: indfall, dtype: int64

In [176]:
csv["indfall"] = csv["indfall"].apply(lambda s: 1 if s == "S" else 0).astype(np.float64)
csv["indfall"] = csv["indfall"].astype(np.int32)

In [177]:
csv.drop("tipodom", axis=1, inplace=True)
csv.drop("cod_prov", axis=1, inplace=True)
csv.drop("nomprov", axis=1, inplace=True)
gc.collect()

31

In [178]:
csv["ind_actividad_cliente"] = csv["ind_actividad_cliente"].astype(np.int32)

In [179]:
csv["renta"].fillna(csv["renta"].median(), inplace=True)
csv["renta"] = csv["renta"].astype(np.float64)

In [180]:
csv["renta"].max()

28894395.510000002

In [181]:
renta_max = 30000000.
csv["renta"] = csv["renta"] / renta_max

In [182]:
segmento_names = [ "01 - TOP", "03 - UNIVERSITARIO", "02 - PARTICULARES"]
segmento_vals = list(range(len(segmento_names)))
csv["segmento"].replace(segmento_names, segmento_vals, inplace=True)

In [183]:
csv["segmento"].fillna(csv["segmento"].median(), inplace=True)
csv["segmento"] = csv["segmento"].astype(np.int32)
gc.collect()

0

In [184]:
# We are interested in those fields
y_fields = []
for c in csv.columns:
    if c.startswith("ind_") and c.endswith("_ult1"):
        y_fields.append(c)
for c in y_fields:
    print(c)
    csv[c] = csv[c].fillna(csv[c].median()).astype(np.int32)
    gc.collect()

ind_ahor_fin_ult1
ind_aval_fin_ult1
ind_cco_fin_ult1
ind_cder_fin_ult1
ind_cno_fin_ult1
ind_ctju_fin_ult1
ind_ctma_fin_ult1
ind_ctop_fin_ult1
ind_ctpp_fin_ult1
ind_deco_fin_ult1
ind_deme_fin_ult1
ind_dela_fin_ult1
ind_ecue_fin_ult1
ind_fond_fin_ult1
ind_hip_fin_ult1
ind_plan_fin_ult1
ind_pres_fin_ult1
ind_reca_fin_ult1
ind_tjcr_fin_ult1
ind_valo_fin_ult1
ind_viv_fin_ult1
ind_nomina_ult1
ind_nom_pens_ult1
ind_recibo_ult1


In [185]:
csv[:5]

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,0,1,1,0.35,0.020000,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-28,1050611,0,1,0,0.23,0.116667,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-28,1050612,0,1,0,0.23,0.116667,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-28,1050613,0,1,1,0.22,0.116667,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0
4,2015-01-28,1050614,0,1,0,0.23,0.116667,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0


In [186]:
# One hot for "ind_empleado"
csv["ind_empleado_N"] = (csv["ind_empleado"] == 0).astype(np.int32)
csv["ind_empleado_B"] = (csv["ind_empleado"] == 1).astype(np.int32)
csv["ind_empleado_F"] = (csv["ind_empleado"] == 2).astype(np.int32)
csv["ind_empleado_A"] = (csv["ind_empleado"] == 3).astype(np.int32)
csv["ind_empleado_S"] = (csv["ind_empleado"] == 4).astype(np.int32)
csv.drop("ind_empleado", axis=1, inplace=True)
gc.collect()

120

In [187]:

csv["indrel_1mes_0"] = (csv["indrel_1mes"] == 0).astype(np.int32)
csv["indrel_1mes_1"] = (csv["indrel_1mes"] == 1).astype(np.int32)
csv["indrel_1mes_2"] = (csv["indrel_1mes"] == 2).astype(np.int32)
csv["indrel_1mes_3"] = (csv["indrel_1mes"] == 3).astype(np.int32)
csv["indrel_1mes_4"] = (csv["indrel_1mes"] == 4).astype(np.int32)
csv["indrel_1mes_5"] = (csv["indrel_1mes"] == 5).astype(np.int32)
csv.drop("indrel_1mes", axis=1, inplace=True)
gc.collect()

49

In [188]:

csv["tiprel_1mes_0"] = (csv["tiprel_1mes"] == 0).astype(np.int32)
csv["tiprel_1mes_1"] = (csv["tiprel_1mes"] == 1).astype(np.int32)
csv["tiprel_1mes_2"] = (csv["tiprel_1mes"] == 2).astype(np.int32)
csv["tiprel_1mes_3"] = (csv["tiprel_1mes"] == 3).astype(np.int32)
csv["tiprel_1mes_4"] = (csv["tiprel_1mes"] == 4).astype(np.int32)
csv.drop("tiprel_1mes", axis=1, inplace=True)
gc.collect()

42

In [189]:
for i in canal_entrada_vals:
    csv["canal_entrada_"+str(i)] = (csv["canal_entrada"] == i).astype(np.int32)
csv.drop("canal_entrada", axis=1, inplace=True)
gc.collect()

99

In [190]:
monthes = np.unique(csv["fecha_dato"])

In [191]:
monthes_vals = list(range(len(monthes)))
csv["fecha_dato"] = csv["fecha_dato"].replace(monthes, monthes_vals).astype(np.int32)

In [192]:
customers = np.unique(csv["ncodpers"])

In [211]:
(customers.shape, len(monthes))

((949614,), 17)

In [205]:
chunk_idx = np.ndarray(shape=(len(customers),len(monthes)))
groups = csv.groupby('ncodpers')
keys = groups.groups.keys()

# for i, row in csv.iterrows():
#     row = csv.iloc[i].values
#     month, cid, data = int(row[0]), int(row[1]), row[2:]
#     print(month, cid, data)
#     break

In [212]:
group_idx = {}
for cid in groups.groups.keys():
    if len(groups.groups[cid]) == len(monthes) :
        group_idx[cid] = groups.groups[cid]

In [213]:
(len(group_idx), len(groups.groups))

(605464, 949614)

In [ ]:
class BatchGenerator(object):
    
    def __init__(self, group_idx, batch_size, unrollings, monthes = 17):
        self.group_idx = group_idx
        self.batch_size = batch_size
        self.unrollings = unrollings
        self.monthes = monthes
        self.month = 0
        self.offset = 0
        self._generator = self._init_generator()
        
    def __next__(self):
        batch = np.zeros(shape=(self.batch_size, self.unrollings), dtype=np.int)
        for i in range(self.batch_size):
            batch[i] = self._forever_next()

        return batch
    
    def _forever_next(self):
        try:
            item = next(self._generator)
            return item
        except StopIteration:
            print("StopIteration, begin again")
            self._generator = self._init_generator()
            item = next(self._generator)
            return item
                           
                           
    def _init_generator(self):
        for i in range(self.monthes - self.unrollings):
            for cid in self.group_idx.keys():
                yield np.array(self.group_idx[cid][i:i+self.unrollings])
    
    
    
        

In [234]:
g = BatchGenerator(group_idx, 16, 6)
print(g.__next__())


[[   1557 1247923 1852436 2505660 2546653 3174749]
 [   1558 1247922 1852435 2505659 2546638 3174750]
 [   1559 1247921 1852434 2505658 2546639 3174751]
 [   1560 1247920 1852433 2505673 2546640 3174752]
 [   1545 1247919 1852432 2505675 2546641 3174753]
 [   1561 1247918 1852477 2505692 2546642 3174754]
 [   1544 1247917 1852488 2505676 2546643 3174755]
 [   1542 1247916 1852554 2505691 2546644 3174756]
 [   1527 1247915 1852489 2505690 2546645 3174757]
 [   1528 1247930 1852542 2505689 2546646 3174758]
 [   1529 1247932 1852541 2505688 2546647 3174759]
 [   1531 1247933 1852539 2505686 2546649 3174707]
 [   1532 1247948 1852538 2505685 2546650 3174772]
 [   1533 1247947 1852537 2505684 2546651 3174706]
 [   1534 1247946 1852536 2505683 2546652 3174695]
 [ 281807  975371 1575575 2202306 2818899 3458697]]


In [284]:
cols = list(csv.columns.values.copy())
cols.remove("fecha_dato")
cols.remove("ncodpers")
X_fields = np.array(cols, dtype="O")

for f in y_fields:
    cols.remove(f)
X_no_y_fields = np.array(cols, dtype="O")

(X_fields, X_no_y_fields)

(array(['pais_residencia', 'sexo', 'age', 'antiguedad', 'indrel',
        'ult_fec_cli_1t', 'indresi', 'indfall', 'ind_actividad_cliente',
        'renta', 'segmento', 'ind_ahor_fin_ult1', 'ind_aval_fin_ult1',
        'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 'ind_cno_fin_ult1',
        'ind_ctju_fin_ult1', 'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1',
        'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 'ind_deme_fin_ult1',
        'ind_dela_fin_ult1', 'ind_ecue_fin_ult1', 'ind_fond_fin_ult1',
        'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 'ind_pres_fin_ult1',
        'ind_reca_fin_ult1', 'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1',
        'ind_viv_fin_ult1', 'ind_nomina_ult1', 'ind_nom_pens_ult1',
        'ind_recibo_ult1', 'ind_empleado_N', 'ind_empleado_B',
        'ind_empleado_F', 'ind_empleado_A', 'ind_empleado_S',
        'indrel_1mes_0', 'indrel_1mes_1', 'indrel_1mes_2', 'indrel_1mes_3',
        'indrel_1mes_4', 'indrel_1mes_5', 'tiprel_1mes_0', 'tiprel_1mes_1',
        'tiprel_1mes_2

In [287]:
(len(X_fields),len(X_no_y_fields),len(y_fields))

(61, 37, 24)

In [262]:

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# Machine Learning

In [302]:

num_unrollings=10

batch_size=32
hidden_size = 64


input_size = len(X_fields)
last_input_size = len(X_no_y_fields)
output_size = len(y_fields)


graph_rnn = tf.Graph()
with graph_rnn.as_default():
    
    # Inputs
    dropout_placeholder = tf.placeholder(tf.float32, name='Dropout')
    # Input data.
    train_inputs = list()
    train_inputs_last = list()
    train_labels = list()
    for _ in range(num_unrollings):
        train_inputs.append(tf.placeholder(tf.float32, shape=[batch_size,input_size]))  # I - I - I - I
        train_labels.append(tf.placeholder(tf.float32, shape=[batch_size,output_size])) #     O - O - O - O
        train_inputs_last.append(tf.placeholder(tf.float32, shape=[batch_size,last_input_size]))
    
    input_placeholder = tf.placeholder(
        tf.float64, shape=[batch_size, input_size], name='Input')
    labels_placeholder = tf.placeholder(
        tf.float64, shape=[batch_size, output_size], name='Target')
    
    with tf.variable_scope('InputDropout'):
        train_inputs = [tf.nn.dropout(x, dropout_placeholder) for x in train_inputs]
    # RNN
    rnn_outputs = []
    with tf.variable_scope('RNN') as scope:
        print("Shape one RNN input", train_inputs[0].get_shape())
        initial_state = tf.zeros([batch_size, hidden_size])
        state = initial_state
        
        for tstep, current_input in enumerate(train_inputs):
            if tstep > 0:
                scope.reuse_variables()
            RNN_H = tf.get_variable('HMatrix', [hidden_size, hidden_size], initializer=tf.truncated_normal_initializer(stddev=0.1))
            RNN_I = tf.get_variable('IMatrix', [input_size, hidden_size], initializer=tf.truncated_normal_initializer(stddev=0.1))
            RNN_b = tf.get_variable('B', [hidden_size], initializer=tf.constant_initializer(0.0))
            
            state = tf.nn.sigmoid(tf.matmul(state, RNN_H) + tf.matmul(current_input, RNN_I) + RNN_b)
            rnn_outputs.append(state)
        final_state = rnn_outputs[-1]
        print("Shape one RNN", final_state.get_shape())
    
    with tf.variable_scope('Last'):
        print("Shape one Last input", train_inputs_last[0].get_shape())
        W_last = tf.get_variable('W_last', [last_input_size, hidden_size], initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_last = tf.get_variable('b_last', [hidden_size], initializer=tf.constant_initializer(0.0))
        tf.add_to_collection('total_loss', tf.nn.l2_loss(W_last))
        
        features_last = [tf.matmul(x, W_last) + b_last for x in train_inputs_last]
        print("Shape one Last", features_last[0].get_shape())
    
    #Projection
    with tf.variable_scope('Projection'):
        concatenated = [tf.concat(1, (features_last[i], rnn_outputs[i])) for i in range(num_unrollings)]
        print("Shape one concatenated (RNN + Last)", concatenated[0].get_shape())
        
        W_proj = tf.get_variable('W_proj', [hidden_size*2, output_size], initializer=tf.truncated_normal_initializer(stddev=0.1))
        b_proj = tf.get_variable('b_proj', [output_size], initializer=tf.constant_initializer(0.0))
        tf.add_to_collection('total_loss', tf.nn.l2_loss(W_proj))
        
        outputs = [tf.matmul(o, W_proj) + b_proj for o in concatenated]
        print("Shape one output", outputs[0].get_shape())
    
    cross_entropy = tf.reduce_mean(
      tf.nn.sigmoid_cross_entropy_with_logits(tf.concat(0, outputs), tf.concat(0, train_labels)))
    tf.add_to_collection('total_loss', cross_entropy)
    
    loss = tf.add_n(tf.get_collection('total_loss'))
    
    optimizer = tf.train.AdamOptimizer(0.0001).minimize(loss)
    
    predictions = [tf.nn.sigmoid(o) for o in outputs]
    
    auc = tf.contrib.metrics.streaming_auc(predictions[-1], train_labels[-1])
    
gc.collect() 

Shape one RNN input (32, 61)
Shape one RNN (32, 64)
Shape one Last input (32, 37)
Shape one Last (32, 37)
Shape one concatenated (RNN + Last) (32, 128)
Shape one output (32, 24)


111090

In [ ]:
num_steps = len(csv)//batch_size//num_unrollings
print("steps", num_steps)

with tf.Session(graph=graph_rnn) as session:
    
    init_op_all = tf.initialize_all_variables()
    init_op_local = tf.initialize_local_variables()
    saver = tf.train.Saver()
    init_op_all.run()
    init_op_local.run()
    print('Initialized')
    
    g = BatchGenerator(group_idx, batch_size, num_unrollings+1)
    
    for i in range(num_steps):
        feed_dict = {dropout_placeholder: 1}
        
        batch_idx = next(g)
        for _i in range(num_unrollings):
            _X = csv.iloc[batch_idx[:,_i]][X_fields].values
            _y = csv.iloc[batch_idx[:,_i+1]][y_fields].values
            _X_no_y = csv.iloc[batch_idx[:,_i+1]][X_no_y_fields].values
            feed_dict[train_inputs[_i]] = _X
            feed_dict[train_inputs_last[_i]] = _X_no_y
            feed_dict[train_labels[_i]] = _y
        
        _ , state, y_pred, l, a = session.run(
            [optimizer, final_state, predictions[-1], loss, auc], feed_dict=feed_dict)
        if i%100 == 0:
            print(i, l, a)
        if i%10000 == 0:
            print(y_pred[0])
            print(_y[0])
    save_path = saver.save(session, "santander.ckpt")

steps 42561
Initialized
0 44.8831 (0.0, 0.29694676)
[ 0.75921685  0.88538045  0.39598748  0.87144881  0.57983309  0.84354663
  0.32116571  0.8620556   0.77205902  0.87459809  0.83310044  0.54807383
  0.32306722  0.1362917   0.40194657  0.4615061   0.30799463  0.29567885
  0.44778401  0.40670407  0.82711935  0.84360921  0.37121505  0.30135393]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
100 39.4164 (0.3255057, 0.32567453)
200 34.6528 (0.37174982, 0.37192219)
300 30.484 (0.38403448, 0.3844007)
400 26.8387 (0.41256714, 0.4124954)
500 23.6293 (0.42655647, 0.42667821)
600 20.8008 (0.4379406, 0.43809837)
700 18.3164 (0.44802818, 0.44812334)
800 16.1094 (0.46167871, 0.46178195)
900 14.1766 (0.47644117, 0.47650564)
1000 12.4762 (0.48841333, 0.48854458)
1100 10.9651 (0.50203037, 0.50218689)
1200 9.64948 (0.51974225, 0.51999235)
1300 8.46707 (0.54199684, 0.54220033)
1400 7.44058 (0.56156391, 0.56173557)
1500 6.53144 (0.57458305, 0.5745846)
1600 5.71062 (0.58420074, 0.58420038)
1700 5.01161